# ov.Agent Ten-Task Regression Suite

This notebook instantiates **ov.Agent** once and then reuses it across ten composite tasks that span end-to-end processing, multi-batch integration, trajectory inference, multiome alignment, tumor microenvironment analytics, and spatial modeling. Each task pairs a carefully worded ov.Agent prompt with lightweight validation hooks so you can quickly benchmark whether the agent still covers the relevant skills.

## Notebook structure

1. Bootstrap ov.Agent with the same diagnostics used in `Tutorials-llm/t_ov_agent_pbmc3k.ipynb`.
2. Load the [`ov_agent_tutorial_data_sources.md`](../../OvIntelligence/ov_agent_tutorial_data_sources.md) manifest so every task can remind you which local `.h5ad` files or remote links must be staged.
3. Define ten task specs (prompt + datasets + keyword checks) and a helper `run_task` wrapper that:
   - prints data reminders via `ensure_data(...)`
   - calls `agent.chat(prompt)`
   - records simple keyword-based health checks in `TASK_LOG`
4. Optionally loop through all specs to stress-test ov.Agent end to end, then summarize pass/fail status in a scoreboard.

In [ ]:
import sys, omicverse as ov
print(sys.executable)
print('omicverse', getattr(ov, '__version__', 'unknown'), ov.__file__)


In [ ]:
import os
from pathlib import Path
import json
from datetime import datetime
from IPython.display import Markdown, display
import scanpy as sc
import omicverse as ov

print('OmicVerse version:', getattr(ov, '__version__', 'unknown'))
print('Supported models:', ov.list_supported_models())

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', '')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY', '')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY', '')

if not (OPENAI_API_KEY or ANTHROPIC_API_KEY or GEMINI_API_KEY):
    print('Warning: set OPENAI_API_KEY / ANTHROPIC_API_KEY / GEMINI_API_KEY before running live ov.Agent requests.')

model_id = 'gpt-5'
api_key = OPENAI_API_KEY or ANTHROPIC_API_KEY or GEMINI_API_KEY
agent = ov.Agent(model=model_id, api_key=api_key)
agent


## Data source manifest helper

`ensure_data(...)` looks up tutorial references inside `OvIntelligence/ov_agent_tutorial_data_sources.md` so that each task can remind you which `.h5ad` or model files should be prepared ahead of time.

In [ ]:
DATA_SOURCE_PATH = Path('OvIntelligence/ov_agent_tutorial_data_sources.md')

def _parse_data_source_table():
    mapping = {}
    if not DATA_SOURCE_PATH.exists():
        return mapping
    current_section = None
    for raw_line in DATA_SOURCE_PATH.read_text().splitlines():
        line = raw_line.strip()
        if line.startswith('## '):
            current_section = line[3:].strip()
            continue
        if not line.startswith('| `Tutorial'):
            continue
        parts = [p.strip() for p in line.strip('|').split('|')]
        if len(parts) < 4:
            continue
        tutorial_ref = parts[0].strip('`')
        mapping[tutorial_ref] = {
            'files': parts[1],
            'size': parts[2],
            'hint': parts[3],
            'section': current_section,
        }
    return mapping

DATA_SOURCES = _parse_data_source_table()
print(f'Loaded {len(DATA_SOURCES)} tutorial rows from {DATA_SOURCE_PATH}')


In [ ]:
def ensure_data(tutorial_refs):
    """Print local-file reminders for the given tutorial references."""
    refs = tutorial_refs
    if isinstance(refs, str):
        refs = [refs]
    display(Markdown('**Data reminders**'))
    for ref in refs:
        info = DATA_SOURCES.get(ref)
        if info:
            display(Markdown(
                f'- `{ref}` (section: {info.get("section", "n/a")}):
'
                f'  - Files/models: {info.get("files", "see tutorial")}
'
                f'  - Approx. size: {info.get("size", "n/a")}
'
                f'  - Source hint: {info.get("hint", "see tutorial")}'
            ))
        else:
            display(Markdown(f'- `{ref}`: not found in manifest — consult the tutorial README.'))


## Task catalog
Each entry specifies a real-world ov.Agent prompt, the tutorial references whose data are required, and lightweight keyword checks. The prompts deliberately cite ov.Agent's skill registry entries instead of notebook filenames so they can be executed directly inside the agent without leaking reference documents.

In [ ]:
TASK_SPECS = [
    {
        'name': 'PBMC 5k/8k FASTQ → QC → cluster-stability benchmarking',
        'datasets': [
            'Tutorials-single/t_alignment_1k.ipynb',
            'Tutorials-single/t_preprocess.ipynb',
            'Tutorials-single/t_cluster.ipynb',
        ],
        'prompt': """You are ov.Agent orchestrating an end-to-end PBMC 5k/8k workflow.

Use the skill registry to load and cite:
- `single-preprocessing` for QC/HVG/scaling guidance.
- `single-clustering` for multi-head resolution sweeps.
- `data-viz-plots` plus `single-downstream-analysis` for stability diagnostics.

Requirements:
1. Start from raw 10x FASTQs, run kb-python alignment to generate `adata.h5ad` counts (show CLI commands).
2. Apply PBMC-grade QC thresholds, normalization, and HVG selection, documenting parameter choices.
3. Run Leiden, Louvain, Gaussian mixture, and LDA clustering across several resolutions, computing UMAP drift metrics that quantify stability.
4. Return ordered code blocks plus a markdown table summarizing head, resolution, drift (0–1), and the recommended resolution.""",
        'checks': [
            {'description': 'Mentions kb-python alignment', 'keywords': ['kb-python', 'FASTQ']},
            {'description': 'Reports UMAP drift or stability table', 'keywords': ['UMAP', 'drift']},
        ],
    },
    {
        'name': 'Pancreas multi-study merge with SIMBA embeddings',
        'datasets': [
            'Tutorials-single/t_single_batch.ipynb',
            'Tutorials-single/t_simba.ipynb',
        ],
        'prompt': """Integrate the Baron, Segerstolpe, and Muraro pancreas scRNA-seq donors under strong batch effects.

Use the registry to load:
- `single-preprocessing` for donor-level normalization and covariate regression.
- `single-multiomics` for SIMBA-style heterogeneous graph construction.
- `single-clustering` and `data-viz-plots` for UMAP diagnostics.

Workflow expectations:
1. Highlight how to harmonize preprocessing parameters across cohorts before building the SIMBA graph.
2. Provide the concrete SIMBA commands that add nodes/edges, train embeddings, and export 2D projections.
3. Quantify integration with before/after UMAPs, kBET, and silhouette scores, commenting on endocrine vs. exocrine separation.
4. Explain how to persist the learned embeddings for downstream classifiers.""",
        'checks': [
            {'description': 'References SIMBA graph embedding', 'keywords': ['SIMBA', 'embedding']},
            {'description': 'Includes kBET or silhouette metrics', 'keywords': ['kBET', 'silhouette']},
        ],
    },
    {
        'name': 'Paul15 hematopoietic trajectories with MetaTiME diagnostics',
        'datasets': [
            'Tutorials-single/t_traj.ipynb',
            'Tutorials-single/t_metatime.ipynb',
        ],
        'prompt': """Reconstruct megakaryocyte vs. lymphoid trajectories on Paul15-like hematopoiesis data.

Use the registry to pull:
- `single-trajectory` for diffusion, PAGA, Palantir/VIA, and MetaTiME checkpoints.
- `single-preprocessing` for QC and normalization.
- `single-downstream-analysis` plus `data-viz-plots` for marker summaries.

Deliverables:
1. Describe preprocessing plus neighborhood graph construction before diffusion/PAGA.
2. Run diffusion maps, Palantir/VIA, and identify root and terminal states with pseudotime ordering.
3. Produce branch marker tables for at least two fates and explain MetaTiME cycle diagnostics that validate the ordering.
4. Return code snippets, saved-figure descriptions, and a markdown list of pseudotime milestones.""",
        'checks': [
            {'description': 'Mentions diffusion/Palantir', 'keywords': ['diffusion', 'Palantir']},
            {'description': 'Cites MetaTiME diagnostics', 'keywords': ['MetaTiME', 'cycling']},
        ],
    },
    {
        'name': 'PBMC Multiome 10k GLUE + MOFA factor discovery',
        'datasets': [
            'Tutorials-single/t_mofa_glue.ipynb',
            'Tutorials-single/t_mofa.ipynb',
        ],
        'prompt': """Perform cross-modal alignment for PBMC Multiome 10k.

Use skill registry lookups for:
- `single-multiomics` (GLUE pairing + MOFA training).
- `single-preprocessing` (modality-specific normalization).
- `data-viz-plots` (factor visualization).

Tasks:
1. Pair RNA and ATAC embeddings with GLUE and report the path to the paired metadata.
2. Train MOFA on matched matrices, labelling shared, RNA-only, and ATAC-only factors with variance explained tables.
3. Highlight at least one IFN-response and one chromatin-accessibility-specific factor, with marker genes/peaks.
4. Provide code snippets plus interpretation bullets for each factor category.""",
        'checks': [
            {'description': 'GLUE pairing mentioned', 'keywords': ['GLUE', 'pair']},
            {'description': 'MOFA factor summaries provided', 'keywords': ['MOFA', 'factor']},
        ],
    },
    {
        'name': 'PBMC 5k scATAC label transfer via GLUE embeddings',
        'datasets': [
            'Tutorials-single/t_anno_trans.ipynb',
        ],
        'prompt': """Transfer PBMC RNA annotations onto PBMC 5k scATAC cells.

Use skill registry entries:
- `single-multiomics` for GLUE-derived embeddings and cross-modal graphs.
- `single-annotation` for label transfer/validation patterns.

Instructions:
1. Load the GLUE embeddings (`data/analysis_lymph/rna-emb.h5ad` and `data/analysis_lymph/atac-emb.h5ad`).
2. Build the cross-modal KNN graph, migrate labels with confidence, and surface per-cluster agreement statistics.
3. Flag potential mismatches and explain how to visualize transferred labels on ATAC UMAPs.
4. Return python commands plus a markdown table of cluster vs. confidence.""",
        'checks': [
            {'description': 'Mentions cross-modal KNN transfer', 'keywords': ['KNN', 'transfer']},
            {'description': 'Reports confidence per cluster', 'keywords': ['confidence', 'cluster']},
        ],
    },
    {
        'name': 'Tumor microenvironment ligand–receptor diagnostics (CellPhoneDBViz)',
        'datasets': [
            'Tutorials-single/t_cellphonedb.ipynb',
        ],
        'prompt': """Contrast ligand–receptor usage between exhausted T cells and M2 macrophages for treated vs. untreated tumors.

Use registry calls for:
- `single-cellphone-db` (interaction formatting, execution, visualization).
- `single-downstream-analysis` and `data-viz-plots` (interpretation + figure export).

Expectations:
1. Show metadata formatting, CellPhoneDB execution commands, and result parsing.
2. Highlight top ligand–receptor pairs per condition with effect sizes/p-values.
3. Describe how to build heatmaps and chord diagrams (include filenames) via CellPhoneDBViz helpers.
4. Provide interpretation guidance on shifts between conditions.""",
        'checks': [
            {'description': 'CellPhoneDB workflow described', 'keywords': ['CellPhoneDB', 'ligand']},
            {'description': 'Mentions visualization artifacts', 'keywords': ['heatmap', 'chord']},
        ],
    },
    {
        'name': 'MetaTiME-driven immune microenvironment annotation',
        'datasets': [
            'Tutorials-single/t_metatime.ipynb',
        ],
        'prompt': """Annotate tumor-infiltrating immune cells with MetaTiME.

Use skill registry lookups for:
- `single-trajectory` (MetaTiME scoring + pseudotime context).
- `single-preprocessing` (optional inferCNV-based malignant removal).
- `single-downstream-analysis` (report formatting).

Deliverables:
1. Optionally filter malignant cells via infercnvpy outputs and recompute neighbors in SCVI space.
2. Run MetaTiME scoring, rank meta-components per cluster, and interpret dominant immune states.
3. Provide preprocessing + scoring code plus a markdown report mapping cluster → top meta-component with interpretation.""",
        'checks': [
            {'description': 'Mentions MetaTiME scoring', 'keywords': ['MetaTiME', 'meta-component']},
            {'description': 'References SCVI neighbors/inferCNV filtering', 'keywords': ['SCVI', 'inferCNV']},
        ],
    },
    {
        'name': 'CEFCON driver regulator discovery',
        'datasets': [
            'Tutorials-single/t_cellfate.ipynb',
        ],
        'prompt': """Discover lineage-specific driver regulators with CEFCON on Nestorowa/Paul15 hematopoiesis.

Use the registry to load:
- `single-trajectory` (fate modeling context).
- `single-downstream-analysis` (regulator reporting).

Steps:
1. Document preprocessing and prior network setup, including how to load the NicheNet graph.
2. Run `ov.single.pyCEFCON`, exporting regulon tables for at least two branches (erythroid vs. granulocyte).
3. Provide tuning advice (walk length, regularization) and show how to visualize regulator activity heatmaps.
4. Summarize key regulators per branch in markdown.""",
        'checks': [
            {'description': 'Mentions CEFCON run + prior network', 'keywords': ['CEFCON', 'NicheNet']},
            {'description': 'Reports regulon or regulator tables', 'keywords': ['regulon', 'regulator']},
        ],
    },
    {
        'name': 'Precision oncology prioritization (inferCNV + scDrug)',
        'datasets': [
            'Tutorials-single/t_scdrug.ipynb',
        ],
        'prompt': """Combine inferCNV-based malignant calling with scDrug predictions for precision oncology.

Use registry lookups for:
- `single-multiomics` (scDrug + modality handling).
- `single-downstream-analysis` (drug ranking summaries).
- `data-viz-plots` (CNV heatmap references).

Workflow:
1. Run infercnvpy to separate malignant vs. normal cells and reference the CNV heatmap artifact.
2. Feed malignant clones into scDrug, compute predicted IC50 values, and rank at least five compounds per clone.
3. Provide code for exporting the ranking table plus guidance on cross-referencing copy-number context when interpreting drug hits.""",
        'checks': [
            {'description': 'inferCNV workflow described', 'keywords': ['inferCNV', 'CNV']},
            {'description': 'scDrug IC50 ranking reported', 'keywords': ['scDrug', 'IC50']},
        ],
    },
    {
        'name': 'Spatial pseudo-time + alignment (SpaceFlow + STAligner)',
        'datasets': [
            'Tutorials-space/t_spaceflow.ipynb',
            'Tutorials-space/t_staligner.ipynb',
        ],
        'prompt': """Analyze Visium DLPFC slices with spatial pseudo-time and cross-slice alignment.

Use the skill registry to load:
- `single-to-spatial-mapping` and `spatial-tutorials` for Visium preprocessing, SpaceFlow, and STAligner steps.
- `data-viz-plots` for reporting aligned layer maps.

Expectations:
1. Run SpaceFlow on slice 151676, computing embeddings, domains, and pseudo-spatiotemporal maps (pSM) from the `151676_filtered_feature_bc_matrix.h5` input.
2. Feed at least two consecutive slices into STAligner with triplet-loss GAT alignment, highlighting how cortical layers align.
3. Report output filenames for aligned embeddings and describe conserved layers across slices in markdown.""",
        'checks': [
            {'description': 'SpaceFlow pseudo-time mentioned', 'keywords': ['SpaceFlow', 'pSM']},
            {'description': 'STAligner alignment described', 'keywords': ['STAligner', 'alignment']},
        ],
    },
]
print(f"{len(TASK_SPECS)} task specs loaded.")


## Task runner utilities

`run_task(...)` displays context, ensures data availability reminders, calls ov.Agent, and stores keyword-check results inside `TASK_LOG` for later summarization. Adjust `RESPONSE_PARSER` if your deployment returns structured JSON instead of plain text.

In [ ]:
TASK_LOG = []

def evaluate_response_text(response_text, checks):
    text = response_text or ''
    lower = text.lower()
    results = []
    for check in checks:
        keywords = [kw.lower() for kw in check.get('keywords', [])]
        passed = all(kw in lower for kw in keywords)
        results.append({
            'description': check.get('description', ''),
            'keywords': keywords,
            'passed': passed,
        })
    return results


def run_task(spec):
    display(Markdown(f"## Task: {spec['name']}"))
    ensure_data(spec.get('datasets', []))
    prompt = spec['prompt']
    response = agent.chat(prompt)
    if isinstance(response, dict):
        response_text = json.dumps(response, indent=2)
    else:
        response_text = str(response)
    checks = evaluate_response_text(response_text, spec.get('checks', []))
    TASK_LOG.append({
        'name': spec['name'],
        'timestamp': datetime.utcnow().isoformat() + 'Z',
        'checks': checks,
        'response_preview': response_text[:2000],
    })
    display(Markdown('**Response preview**'))
    display(Markdown(response_text[:2000]))
    display(Markdown('**Check results**'))
    for chk in checks:
        status = '✅' if chk['passed'] else '❌'
        display(Markdown(f"- {status} {chk['description']} (keywords: {', '.join(chk['keywords'])})"))
    return response


## Execute the full suite

Set `RUN_ALL = True` to launch all ten prompts sequentially. You can also call `run_task(TASK_SPECS[i])` manually for spot checks.

In [ ]:
RUN_ALL = False

if RUN_ALL:
    for spec in TASK_SPECS:
        run_task(spec)


### Task 1: PBMC 5k/8k FASTQ → QC → cluster-stability benchmarking
Run this cell to execute only this scenario. The helper below will print data reminders and log the ov.Agent response.

In [ ]:
run_task(TASK_SPECS[0])

### Task 2: Pancreas multi-study merge with SIMBA embeddings
Run this cell to execute only this scenario. The helper below will print data reminders and log the ov.Agent response.

In [ ]:
run_task(TASK_SPECS[1])

### Task 3: Paul15 hematopoietic trajectories with MetaTiME diagnostics
Run this cell to execute only this scenario. The helper below will print data reminders and log the ov.Agent response.

In [ ]:
run_task(TASK_SPECS[2])

### Task 4: PBMC Multiome 10k GLUE + MOFA factor discovery
Run this cell to execute only this scenario. The helper below will print data reminders and log the ov.Agent response.

In [ ]:
run_task(TASK_SPECS[3])

### Task 5: PBMC 5k scATAC label transfer via GLUE embeddings
Run this cell to execute only this scenario. The helper below will print data reminders and log the ov.Agent response.

In [ ]:
run_task(TASK_SPECS[4])

### Task 6: Tumor microenvironment ligand–receptor diagnostics (CellPhoneDBViz)
Run this cell to execute only this scenario. The helper below will print data reminders and log the ov.Agent response.

In [ ]:
run_task(TASK_SPECS[5])

### Task 7: MetaTiME-driven immune microenvironment annotation
Run this cell to execute only this scenario. The helper below will print data reminders and log the ov.Agent response.

In [ ]:
run_task(TASK_SPECS[6])

### Task 8: CEFCON driver regulator discovery
Run this cell to execute only this scenario. The helper below will print data reminders and log the ov.Agent response.

In [ ]:
run_task(TASK_SPECS[7])

### Task 9: Precision oncology prioritization (inferCNV + scDrug)
Run this cell to execute only this scenario. The helper below will print data reminders and log the ov.Agent response.

In [ ]:
run_task(TASK_SPECS[8])

### Task 10: Spatial pseudo-time + alignment (SpaceFlow + STAligner)
Run this cell to execute only this scenario. The helper below will print data reminders and log the ov.Agent response.

In [ ]:
run_task(TASK_SPECS[9])

## Scoreboard

After running one or more tasks, execute the cell below to summarize keyword checks and preview truncated responses.

In [ ]:
from pprint import pprint

if not TASK_LOG:
    print('No tasks have been executed yet.')
else:
    summary = []
    for entry in TASK_LOG:
        summary.append({
            'name': entry['name'],
            'checks_passed': sum(1 for chk in entry['checks'] if chk['passed']),
            'checks_total': len(entry['checks']),
        })
    pprint(summary)
